In [ ]:
from pathlib import Path

def find_project_root(start: Path = Path.cwd(),
                      required_subpath: Path = Path('Data') / 'Normal' / 'Ripe') -> Path:
    """
    Durchläuft start und alle Elternverzeichnisse und gibt das erste zurück,
    in dem required_subpath existiert.
    """
    for p in [start] + list(start.parents):
        if (p / required_subpath).is_dir():
            return p
    raise FileNotFoundError(f"Kein Verzeichnis gefunden, das '{required_subpath}' enthält (Beginn: {start})")

# Suche das Projekt-Root anhand des existierenden Data/Normal/Ripe-Pfads
BASE_DIR = find_project_root()
print(f"✅ Projekt-Root gefunden: {BASE_DIR}")

# Jetzt geht’s weiter wie gehabt:
image_dir = BASE_DIR / 'Data' / 'Normal' / 'Ripe'
label_dir = BASE_DIR / 'labels' / 'Ripe'

# Sollte jetzt nicht mehr fehlschlagen:
label_dir.mkdir(parents=True, exist_ok=True)

class_id = 0  # Klasse 0 = "ripe"
count = 0

for img_path in image_dir.iterdir():
    if img_path.suffix.lower() in {'.jpg', '.jpeg', '.png'}:
        txt_path = label_dir / f"{img_path.stem}.txt"
        with open(txt_path, 'w') as f:
            f.write(f"{class_id} 0.5 0.5 1.0 1.0\n")
        count += 1

print(f"✅ {count} Labels erstellt in {label_dir}")


✅ Basis-Verzeichnis gefunden: c:\Users\Maxi\Documents\Forschsem\Erdbeeren\Versuch_2\Erdbeeren_yolo\Riseholme


FileNotFoundError: c:\Users\Maxi\Documents\Forschsem\Erdbeeren\Versuch_2\Erdbeeren_yolo\Riseholme\Data\Normal\Ripe existiert nicht

datensatz splitten

In [ ]:
from pathlib import Path
from sklearn.model_selection import train_test_split
import shutil

BASE_DIR    = Path().resolve()                           # …/Riseholme-2021
IMG_SRC     = BASE_DIR / 'Data' / 'Normal' / 'Ripe'      # deine Original-Bilder
LBL_SRC     = BASE_DIR / 'labels' / 'Ripe'               # deine erzeugten .txt

# Zielordner für YOLO
IMG_TRAIN   = BASE_DIR / 'images' / 'train'
IMG_VAL     = BASE_DIR / 'images' / 'val'
LBL_TRAIN   = BASE_DIR / 'labels' / 'train'
LBL_VAL     = BASE_DIR / 'labels' / 'val'

# Ordner anlegen
for d in (IMG_TRAIN, IMG_VAL, LBL_TRAIN, LBL_VAL):
    d.mkdir(parents=True, exist_ok=True)

# Dateinamen (Stem) sammeln und splitten
files = [f.stem for f in IMG_SRC.iterdir() if f.suffix.lower() in {'.png','.jpg'}]
train, val = train_test_split(files, test_size=0.2, random_state=42)

# Kopieren
for f in train:
    shutil.copy(IMG_SRC/ f"{f}.png", IMG_TRAIN/ f"{f}.png")
    shutil.copy(LBL_SRC/ f"{f}.txt", LBL_TRAIN/ f"{f}.txt")
for f in val:
    shutil.copy(IMG_SRC/ f"{f}.png", IMG_VAL/ f"{f}.png")
    shutil.copy(LBL_SRC/ f"{f}.txt", LBL_VAL/ f"{f}.txt")

print(f"Train: {len(train)} | Val: {len(val)}")


In [ ]:
from ultralytics import YOLO
model = YOLO('yolov8n.pt')             # oder yolov8s.pt, -m.pt etc.
model.train(
    data='data.yaml',                  # deine Konfig
    epochs=50,                         # je nach Bedarf
    imgsz=640,
    batch=16,
    augment=True,
    project='runs/train',              # Ausgabeordner
    name='ripe_only'                   # Lauf-Name
)
